In [1]:
%matplotlib widget

Trying to replicate the following research piece:
https://www.federalreserve.gov/econres/notes/feds-notes/index-of-common-inflation-expectations-20200902.htm
Essentially to create an indicator of inflation expectations that reads through the noise of various forward looking metrics.

In [2]:
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from fredapi import Fred as FredApi

fred = FredApi(api_key='0dfcbb403eea7fee006f01e935be9216')

#### Survey of Professional Forecasters data
Probably the easiest solution is to download data in xlsx format from the university of Michigan data area located at the below and subpages:

- https://www.philadelphiafed.org/surveys-and-data/real-time-data-research/survey-of-professional-forecasters
- https://www.philadelphiafed.org/surveys-and-data/real-time-data-research/inflation-forecasts

Save the files in this folder with appropriate names.
Once you've got those, you have 7 of the 21 series proposed by the article as they use:

* CPI: 1y ahead, 6-10y ahead, next 10y
* PCE: 1y ahead, 6-10y ahead, next 10y
* Core PCE 1y ahead

There is also another 10y ahead from other sources... so why not!
Notably, there may be differences in what they consider as "6-10y". Maybe they consider the implied 5y5y forward that is also available (and calculated from the 5y and the 10y clearly). 

In [3]:
spf_base = pd.read_excel("spf_meanLevel.xlsx", sheet_name=["CPI","PCE","CPI5YR","CPI10","PCE5YR","PCE10"], header=0);
pce = spf_base["PCE"]
cpi = spf_base["CPI"]
cpi_5y = spf_base["CPI5YR"]
cpi_10y = spf_base["CPI10"]
pce_5y = spf_base["PCE5YR"]
pce_10y = spf_base["PCE10"]

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [4]:
cpi.tail()

,YEAR,QUARTER,CPI1,CPI2,CPI3,CPI4,CPI5,CPI6,CPIA,CPIB,CPIC
209,2021.0,1.0,2.2015,2.4848,2.0349,2.1578,2.2153,2.2171,2.2187,2.1804,2.2727
210,2021.0,2.0,3.7027,3.3839,2.6984,2.4069,2.4551,2.2981,3.0494,2.3902,2.4086
211,2021.0,3.0,8.4037,5.0886,2.6664,2.4430,2.3939,2.4739,4.9558,2.5397,2.4597
212,2021.0,4.0,6.6018,4.6157,3.2337,2.7242,2.6236,2.4932,5.8236,2.7679,2.4772
213,2022.0,1.0,8.2024,5.5158,3.6657,3.0589,3.0151,2.8363,3.7886,2.7123,2.5483


In [5]:
# calculate the 1y ahead forecasts as indicated by the paper PDF in the folder
def one_year_forecast(df, lbl):
    df2 = pd.DataFrame(index=df.index, columns=["YEAR","QUARTER",lbl+"1YR"])
    df2[lbl+"1YR"] = 100*(((1+df[lbl+"3"]/100)*(1+df[lbl+"4"]/100)*(1+df[lbl+"5"]/100)*(1+df[lbl+"6"]/100))**0.25 - 1)
    df2[["YEAR","QUARTER"]] = df[["YEAR","QUARTER"]]
    return df2

cpi_1y = one_year_forecast(cpi, "CPI")
pce_1y = one_year_forecast(pce, "PCE")

In [6]:
# now create an index based on the years and quarters by working on a separate temporary dataframe
df_tmp = cpi[["YEAR","QUARTER"]].copy()
df_tmp["Year"] = df_tmp["YEAR"].astype(int)
df_tmp["Quarter"] = df_tmp["QUARTER"].astype(int)
df_tmp["Month"] = df_tmp["Quarter"]*3
df_tmp["Day"] = 1
df_tmp["Date"] = pd.to_datetime(df_tmp[["Year","Month","Day"]]) + pd.offsets.QuarterEnd(0)
df_tmp.set_index("Date",inplace=True)
# df_tmp.tail()

In [7]:
# now create the final dataframe
# and add the implied 5y5y fwd calculations based on the 5y and the 10y numbers (1+5y)^5 * (1+5y5y)^5 == (1+10y)^10
df_idx = pd.date_range(start=df_tmp.index.min(), end=df_tmp.index.max(), freq='Q')
df = pd.DataFrame(index=df_idx)
df["spf_cpi1y"] = cpi_1y["CPI1YR"].to_numpy()
df["spf_cpi5y"] = cpi_5y["CPI5YR"].to_numpy()
df["spf_cpi10y"] = cpi_10y["CPI10"].to_numpy()
df["spf_cpi5y5y"] =( ((1 + df["spf_cpi10y"]/100)**10 / (1 + df["spf_cpi5y"]/100)**5)**0.2 - 1)*100
df["spf_pce1y"] = pce_1y["PCE1YR"].to_numpy()
df["spf_pce5y"] = pce_5y["PCE5YR"].to_numpy()
df["spf_pce10y"] = pce_10y["PCE10"].to_numpy()
df["spf_pce5y5y"] =( ((1 + df["spf_pce10y"]/100)**10 / (1 + df["spf_pce5y"]/100)**5)**0.2 - 1)*100
df.tail()

,spf_cpi1y,spf_cpi5y,spf_cpi10y,spf_cpi5y5y,spf_pce1y,spf_pce5y,spf_pce10y,spf_pce5y5y
2021-03-31,2.156248,2.2144,2.2384,2.262406,1.936813,1.9861,2.0697,2.153369
2021-06-30,2.464520,2.4445,2.3532,2.261981,2.259901,2.2852,2.2082,2.131258
2021-09-30,2.494248,2.7574,2.4615,2.166452,2.354146,2.4780,2.2445,2.011532
2021-12-31,2.768292,2.9800,2.5814,2.184343,2.506196,2.5954,2.3062,2.017815
2022-03-31,3.143527,2.7698,2.4901,2.211161,2.770849,2.4836,2.2504,2.017731


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 214 entries, 1968-12-31 to 2022-03-31
Freq: Q-DEC
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   spf_cpi1y    163 non-null    float64
 1   spf_cpi5y    67 non-null     float64
 2   spf_cpi10y   122 non-null    float64
 3   spf_cpi5y5y  67 non-null     float64
 4   spf_pce1y    61 non-null     float64
 5   spf_pce5y    61 non-null     float64
 6   spf_pce10y   61 non-null     float64
 7   spf_pce5y5y  61 non-null     float64
dtypes: float64(8)
memory usage: 15.0 KB


In [9]:
df.to_pickle('spf_dataset.pd.pkl')